### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow for calcium imaging, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
# Runs in about 2m
import datajoint as dj
import datetime
import numpy as np
from workflow_calcium_imaging.pipeline import subject, session, scan, imaging, Equipment
from element_calcium_imaging import imaging_report
import suite2p

In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    )
)

In [ ]:
Equipment.insert1(dict(scanner="Mesoscope"))

In [ ]:
session_key = dict(subject='subject1', 
                   session_datetime=datetime.datetime(2023, 5, 11, 12, 00, 00))

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject1/session1'
    )
)

In [ ]:
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=0,
        scanner="Mesoscope",
        acq_software='ScanImage',
    )
)

In [ ]:
# Runs in about 35s
populate_settings = {"display_progress": True}

scan.ScanInfo.populate(**populate_settings)

In [ ]:
suite2p_params = suite2p.default_ops()
suite2p_params['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=suite2p_params,
    paramset_desc='Default parameter set for suite2p'
)

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir='subject1/session1/suite2p',
    )
)

imaging.Processing.populate(**populate_settings)

In [ ]:
# Runs in about 3m
imaging.Curation.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time=datetime.datetime(2023, 5, 11, 12, 00, 00),
        curation_output_dir='subject1/session1/suite2p',
        manual_curation=False,
    )
)

imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)
imaging_report.ScanLevelReport.populate(**populate_settings)
imaging_report.TraceReport.populate(**populate_settings)

### Drop schemas

- Schemas are not typically dropped in a production workflow with real data in it.
- At the developmental phase, it might be required for the table redesign.
- When dropping all schemas is needed, the following is the dependency order.

In [ ]:
def drop_databases(databases):
    import pymysql.err
    conn = dj.conn()

    with dj.config(safemode=False):
        for database in databases:
            schema = dj.Schema(f'{dj.config["custom"]["database.prefix"]}{database}')
            while schema.list_tables():
                for table in schema.list_tables():
                    try:
                        conn.query(f"DROP TABLE `{schema.database}`.`{table}`")
                    except pymysql.err.OperationalError:
                        print(f"Can't drop `{schema.database}`.`{table}`. Retrying...")
            schema.drop()

# drop_databases(databases=['analysis', 'trial', 'event', 'imaging_report', 'imaging', 'scan', 'session', 'subject', 'lab', 'reference'])